In [1]:
from torch.utils.data import *
from lircst_ana_dataset import LircstAnaDataset
from torch import Generator

dataset = LircstAnaDataset('/home/samnub/dev/lircst-ana/data/')

rand_generator = Generator().manual_seed(42) # The meaning of life, the universe and everything

dataset_train, dataset_valid, dataset_test = random_split(dataset, [0.8, 0.1, 0.1], generator=rand_generator)

In [2]:
import pytorch_lightning as pl
from lircst_ana_dataset import LircstAnaDataset
from encoded_conditional_latent_diffusion import ECLDiffusion
from Diffusion.EMA import EMA
from Diffusion.DenoisingDiffusionProcess.samplers import DDIM_Sampler
from Diffusion.DenoisingDiffusionProcess.samplers import DDPM_Sampler


model = ECLDiffusion(
    train_dataset=dataset_train,
    valid_dataset=dataset_valid,
    test_dataset=dataset_test,
    lr=1e-4,
    batch_size=16,
)

# Load the model from a checkpoint
# Comment out the line below if you want to start training from scratch
model = ECLDiffusion.load_from_checkpoint("/home/samnub/dev/lircst-diffusion/src/lightning_logs/version_18/checkpoints/epoch=172-step=200000.ckpt",
                                         train_dataset=dataset_train,
                                         valid_dataset=dataset_valid,
                                         test_dataset=dataset_test,
                                         lr=1e-4,
                                         batch_size=16,)


# Seems like the inference-time DDIM sampler that they provided is bugged
ddpm_sampler=DDPM_Sampler(model.model.train_timesteps)
model.model.diffusion_process.sampler = ddpm_sampler


trainer = pl.Trainer(
    max_steps=2e5,
    callbacks=[EMA(0.9999)],
    accelerator='gpu',
    devices=[0],
    check_val_every_n_epoch=10,
)

/home/samnub/anaconda3/envs/lircst-diffusion/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Is Time embed used ?  True
Is Time embed used ?  True


/home/samnub/dev/lircst-diffusion/src/Diffusion/EMA.py:58: UserWarning: EMA has better performance when Apex is installed: https://github.com/NVIDIA/apex#installation.
  rank_zero_warn(
You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [ ]:
# Train an untrained model

trainer.fit(model)

In [ ]:
# Evaluate the model

trainer.test(model)

You are using a CUDA device ('NVIDIA RTX A4000') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0:   1%|          | 1/116 [00:12<24:23,  0.08it/s]

/home/samnub/anaconda3/envs/lircst-diffusion/lib/python3.13/site-packages/pytorch_lightning/utilities/data.py:79: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 16. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.


Batch 1: PSNR: 6.3975, SSIM: -0.0036
Testing DataLoader 0:  10%|█         | 12/116 [06:33<56:46,  0.03it/s]